In [2]:
!pip install --upgrade pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install --upgrade numpy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np 
import pandas as pd

In [12]:
!pip install --upgrade nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install "numpy>=1.19.5,<1.27.0"

   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ----- ---------------------------------- 2.4/15.8 MB 12.2 MB/s eta 0:00:02
   ------------ --------------------------- 5.0/15.8 MB 11.6 MB/s eta 0:00:01
   ----------------- ---------------------- 7.1/15.8 MB 11.5 MB/s eta 0:00:01
   --------------------------- ------------ 10.7/15.8 MB 11.2 MB/s eta 0:00:01
   ------------------------------------- -- 14.7/15.8 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------  15.7/15.8 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
transformers 2.1.1 requires sentencepiece, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [9]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\JAIPUR\AppData\Roaming\nltk_data...


True

# Define a function to fetch data through SQL query

In [27]:
import pandas as pd
def fetchDataFromSQL(): 
    connection_str = (
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=KAVYA14;"
        "Database=Marketing_Campaign_Effectiveness_Analysis;"
        "Trusted_Connection=yes;" # Use Windows Authentication for the connection
    )
    connection = pyodbc.connect(connection_str)
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM customer_reviews"
    df = pd.read_sql(query, connection)
    connection.close()
    return df
customer_Reviews_df = fetchDataFromSQL()
# Create a SentimentIntensityAnalyzer object.
SIA = SentimentIntensityAnalyzer()


C:\Users\JAIPUR\AppData\Local\Temp\ipykernel_19104\3855215768.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


##Define a function to calculate sentiment scores using VADER (Valence Aware Dictionary and sEntiment Reasoner)

In [43]:
def sentiment_scores(reviewText):
    # polarity_scores method of SentimentIntensityAnalyzer object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment = SIA.polarity_scores(reviewText)
    return sentiment['compound'] # return the compound score b/w -1 to 1 
    

# Define a function to categorize sentiment using both the sentiment score and the review rating

In [51]:
def categorizeSentiment(score,rating):
    #positive sentiment
    if score > 0.05:
        if rating >=4:
            return 'Positive' # High rating but positive sentiment
        elif rating==3:
            return 'Mixed Positive' # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'# low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment
      

In [53]:
def sentimentBucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

# Apply sentiment analysis to calculate sentiment scores for each review
customer_Reviews_df['SentimentScore'] = customer_Reviews_df['ReviewText'].apply(sentiment_scores)

# Apply sentiment categorization using both text and rating
customer_Reviews_df['SentimentCategory'] = customer_Reviews_df.apply(
    lambda row: categorizeSentiment(row['SentimentScore'], row['Rating']), axis=1)## function will be applied to each row

# Apply sentiment bucketing to categorize scores into defined ranges
customer_Reviews_df['SentimentBucket'] = customer_Reviews_df['SentimentScore'].apply(sentimentBucket)

# Display the first few rows of the DataFrame with sentiment scores, categories, and buckets
print(customer_Reviews_df.head())

# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file
customer_Reviews_df.to_csv('customer_reviews_with_sentiment.csv', index=False)

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  
